### Masternode NFT

In [1]:
import os
import copy
import numpy as np
import time
import matplotlib.pyplot as plt
cwd =  os.getcwd().replace("notebooks/simulation","")
os.chdir(cwd)

from python.dev.helper import Helper

from python.dev.simulation import SimulationOrchestrator
from python.dev.simulation import EventQueue
from python.dev.simulation import EventExecutor

from python.dev.agent import TokenAgent
from python.dev.agent import LPTokenAgent
from python.dev.agent import UserAgent

from python.dev.action import LPXDepositAction
from python.dev.action import LPYDepositAction
from python.dev.action import MintAction
from python.dev.event import Mint
from python.dev.event import Deposit

from python.dev.simulation.batch import WithdrawBatch
from python.dev.simulation.batch import DepositBatch
from python.dev.simulation.batch import LPDepositBatch
from python.dev.simulation.batch import SwapBatch
from python.dev.simulation.batch.set import BatchSet

from python.dev.math.model import TokenDeltaModel
from python.dev.math.model import TimeDeltaModel

from python.dev.lp import LiquidityPool
from python.dev.lp.event import LPEvent
from python.dev.lp.event import MintLPEvent
from python.dev.lp.event import DepositLPEvent
from python.dev.lp.event import SwapLPEvent

### Initialize Agents

In [2]:
USER1 = UserAgent('USER1')
SYS = TokenAgent('SYS')
DAI = TokenAgent('DAI')
USER2 = UserAgent('USER2')

SYS_DAI_LP = LiquidityPool('SYS/DAI', USER1, SYS, DAI)
SYS_DAI = LPTokenAgent('SYS_DAI_LP', SYS_DAI_LP)

### Mint

In [3]:
apy = 0.1

In [4]:
SYS_mint1 = Mint(5000, apy, False)
DAI_mint1 = Mint(5000, apy, True)
SYS_DAI_LP_mint1 = Mint(0, apy, True)
SYS_mint2 = Mint(100000, apy, False)
DAI_mint2 = Mint(0, apy, True)
SYS_DAI_LP_mint2 = Mint(0, apy, True)

### Actions

In [5]:
deposit_event1 = Deposit(apy, 5000, 0)
deposit_event2 = Deposit(apy, 5000, 0)

In [6]:
deposit_action1 = LPXDepositAction(deposit_event1, SYS_DAI, USER1)
deposit_action2 = LPYDepositAction(deposit_event2, SYS_DAI, USER1) 

### Batchset